# Jupyter Widgets

In [16]:
from ipywidgets import FileUpload, Button, Output
from IPython.display import display, FileLink
import subprocess

output = Output()
fileInput = FileUpload()
convertButton = Button(description='Convert')

def convert(b):
    with output:
        print('Your PDF will be ready right away. ')
        nbName = list(fileInput.value.keys())[0]
        pdfName = nbName.replace('ipynb', 'pdf')
        with open(nbName, 'w+b') as f:
            f.write(fileInput.data[0]) 
        proc = subprocess.run(['jupyter', 'nbconvert', nbName, '--to', 'pdf'])
        if proc.returncode == 1:
            print('LaTeX compiling error!')
        else:
            print('PDF is ready! You may now download it through the below link: ')
            display(FileLink(pdfName))

convertButton.on_click(convert)

display(fileInput, convertButton, output) 

FileUpload(value={}, description='Upload')

Button(description='Convert', style=ButtonStyle())

Output()

In [19]:
import os

os.getenv('HOST', '127.0.0.1'), os.getenv('PORT', '8050')

('x86_64-conda_cos6-linux-gnu', '8050')